In [116]:
import requests
import requests.auth
from requests_oauthlib import OAuth1
import oauth2 as oauth
import time
import os
import json
import pprint
import csv
import base64
from datetime import datetime

In [117]:
# Import credentials from local source:
reader = csv.reader(open("/Users/mchifala/Desktop/ATLS_5412/Credentials.csv"))
credentials = {}
for line in reader:
    credentials[line[0]] = line[1]
    

In [118]:
# Get a Reddit credentials:
def getRedditHeaders():
    reddit_id = credentials['reddit_id']
    reddit_secret = credentials['reddit_secret']
    reddit_username = credentials['reddit_username']
    reddit_password = credentials['reddit_password']
    reddit_user_agent = credentials['reddit_user_agent']

    reddit_auth = requests.auth.HTTPBasicAuth(reddit_id, reddit_secret)
    reddit_post_data = {'grant_type': 'password', 'username': reddit_username, 'password': reddit_password}
    reddit_token_headers = {'User-Agent': reddit_user_agent}
    r_auth = requests.post("https://www.reddit.com/api/v1/access_token", auth= reddit_auth, data=reddit_post_data, headers=reddit_token_headers)

    reddit_token = r_auth.json()['token_type'] + ' ' + r_auth.json()['access_token']
    
    # Return header dictionary
    return {"Authorization": reddit_token, "User-Agent": reddit_user_agent, 'Content-Type': 'application/json'}


In [119]:
# Get Twitter credentials:
def getTwitterAuth():
    twitter_key = credentials['twitter_key']
    twitter_secret =  credentials['twitter_secret']
    twitter_token = credentials['twitter_token']
    twitter_secret_token = credentials['twitter_secret_token']
    
    #Return authorization object
    return OAuth1(twitter_key, twitter_secret, twitter_token, twitter_secret_token)

getTwitterAuth()

In [120]:
# Get all available Twitter trends:
twitter_auth = getTwitterAuth()
r_trends = requests.get('https://api.twitter.com/1.1/trends/available.json', auth = twitter_auth)

In [121]:
# Extract the location ID codes for trends:
world_codes = []
for i in range(0,len(r_trends.json())):
    world_codes.append(r_trends.json()[i]['woeid'])

test_codes = world_codes[20:30]
print(test_codes)

[26042, 26062, 26734, 28218, 28869, 30079, 30720, 32185, 32452, 32566]


In [122]:
#Search each location ID for its specific trends:
topics = []
for code in world_codes:
    r_trends2 = requests.get('https://api.twitter.com/1.1/trends/place.json?id='+str(code), auth = twitter_auth)
    for i in range(0,len(r_trends2.json()[0])):
        print(r_trends2.json()[0]['trends'][i]['name'])
        topics.append((r_trends2.json()[0]['trends'][i]['name']))

Zidane
#السعوديه_تلغي_احتكار_bein
#NationalNappingDay
Landon Collins
#NationalNappingDay
#MondayMotivation
Landon Collins
Zidane
#CSW63
#CommonwealthDay
#ConnectedCommonwealth
#NationalNappingDay
#NationalNappingDay
#MondayMotivation
Landon Collins
Zidane
#NationalNappingDay
#MondayMotivation
Landon Collins
OECD
#NationalNappingDay
OECD
#MondayMotivation
Belleville
#NationalNappingDay
#MondayMotivation
Landon Collins
Zidane
#NationalNappingDay
#MondayMotivation
Landon Collins
OECD
#BBCAN7
#NationalNappingDay
#MondayMotivation
Landon Collins
#avfc
Grealish
Cheltenham
#bcfc
Liverpool
#SewAngelicThreads
#UniversityChallenge
#Dispatches
Arsenal
Cheltenham
Grealish
Captain Marvel
#bhafc
#UniversityChallenge
#Dispatches
#onlyconnect
#AfterLife
#UniversityChallenge
#onlyconnect
Zidane
#rovers
#WWEFastlane
#DancingOnIce
Arsenal
Grealish
#MondayMotivation
United
Arsenal
Grealish
Birmingham
Liverpool
#AfterLife
#UniversityChallenge
#Dispatches
#onlyconnect
Zidane
#AskLenny
Zidane
#MondayMotivati

KeyError: 0

In [ ]:
print(topics)

In [61]:
def RedditSearch(reddit_headers, elastic_headers, elastic_index, ratelimit_calls, ratelimit_reset, trends):
    for query in set(trends):
        if (int(float(ratelimit_calls)) > 0):
            #print('Before call')
            #print(ratelimit_calls, ratelimit_reset)
            ratelimit_calls, ratelimit_reset = RedditQuery(reddit_headers, elastic_headers, elastic_index, ratelimit_calls, ratelimit_reset, query)  
            #print('After call')
            #print(ratelimit_calls, ratelimit_reset)
            #print(type(ratelimit_calls), type(ratelimit_reset))
        else:
            print('Sleep')
            time.sleep(int(ratelimit_reset))
            ratelimit_calls, ratelimit_reset = RedditQuery(reddit_headers, elastic_headers, elastic_index, ratelimit_calls, ratelimit_reset, query)

In [114]:
def RedditQuery(reddit_headers,elastic_headers,elastic_index, ratelimit_calls, ratelimit_reset, query):
    r_get = requests.get('https://oauth.reddit.com/r/all/search?q='+query+'&t=day&limit=25', headers = reddit_headers)
    posts = r_get.json()['data']['children']
    
    data = []
    fields_to_keep = ['id','title', 'author', 'score', 'subreddit', 'num_comments', 'created_utc', 'url']
    for post in posts:
        temp = {k: post['data'][k] for k in fields_to_keep}
        temp.update({'hashtag': query, 'source': 'reddit'})
        temp['post_date'] = datetime.isoformat(datetime.utcfromtimestamp(temp['created_utc']))
        temp['upvotes'] = temp['score']
        data.append({'index': {"_index": elastic_index, '_type': '_doc', '_id': 'r_'+ temp['id'] }})
        url_split = temp['url'].split('.')[-1]
        if (url_split != "jpg" and url_split != "png"):
            del temp['url']
        del temp['id']
        del temp['created_utc']
        del temp['score']
        data.append(temp)

    if data:
        data_to_post = '\n'.join(json.dumps(d) for d in data)
        print(data_to_post)
        #r_post = requests.post('http://34.73.60.209:9200/'+elastic_index+'/_bulk', headers = elastic_headers, data=data_to_post+'\n')
        #print(r_post.text)
    
    #print(r_get.headers['x-ratelimit-remaining'], r_get.headers['x-ratelimit-reset'])
    
    return r_get.headers['x-ratelimit-remaining'], r_get.headers['x-ratelimit-reset']

In [115]:
# Testing Enviroment for Rate Limits:
reddit_headers = getRedditHeaders()
elastic_headers = {'Content-Type':'application/json'}
elastic_index = 'hi_yash2'

r_get = requests.get('https://oauth.reddit.com', headers = reddit_headers)
ratelimit_calls =  r_get.headers['x-ratelimit-remaining']
ratelimit_reset = r_get.headers['x-ratelimit-reset']

RedditSearch(reddit_headers, elastic_headers, elastic_index, ratelimit_calls, ratelimit_reset, topics)

{"index": {"_index": "hi_yash2", "_type": "_doc", "_id": "r_azjypw"}}
{"title": "Iglesia cat\u00f3lica ... la en\u00e9sima asignatura suspensa de Periquito S\u00e1nchez", "author": "LeopoldoSalgui", "subreddit": "podemos", "num_comments": 0, "hashtag": "Cat\u00f3lica", "source": "reddit", "post_date": "2019-03-10T20:24:20", "upvotes": 2}
{"index": {"_index": "hi_yash2", "_type": "_doc", "_id": "r_azkevs"}}
{"title": "S\u00e9rio - Opini\u00e3o Mestrado Nova IMS", "author": "Terrible_At_Parking", "subreddit": "portugal", "num_comments": 0, "hashtag": "Cat\u00f3lica", "source": "reddit", "post_date": "2019-03-10T21:03:58", "upvotes": 5}
{"index": {"_index": "hi_yash2", "_type": "_doc", "_id": "r_azl451"}}
{"title": "Catherine Fournier fera le point sur son avenir politique lundi", "author": "Bienvenue12", "subreddit": "Quebec", "num_comments": 15, "hashtag": "Catherine Fournier", "source": "reddit", "post_date": "2019-03-10T22:05:51", "upvotes": 9}
{"index": {"_index": "hi_yash2", "_type"

{"index": {"_index": "hi_yash2", "_type": "_doc", "_id": "r_azlrqa"}}
{"title": "Nuestro Padre Jes\u00fas de las Penas de la Hermandad de las Penas de San Vicente en La Campana, a la vuelta de su Estaci\u00f3n de Penitencia a la Santa Iglesia Catedral (Lunes Santo del a\u00f1o 2018)", "author": "SiAustralia16", "subreddit": "SemanaSantaSevillana", "num_comments": 0, "hashtag": "Campana", "source": "reddit", "post_date": "2019-03-10T23:06:40", "upvotes": 1}
{"index": {"_index": "hi_yash2", "_type": "_doc", "_id": "r_azetjf"}}
{"title": "ABSOLUTELY CORRUPT. SAD!", "author": "byeserenasaves", "subreddit": "The_Donald", "num_comments": 523, "url": "http://magaimg.net/img/7gao.png", "hashtag": "Absolutely", "source": "reddit", "post_date": "2019-03-10T11:30:09", "upvotes": 10319}
{"index": {"_index": "hi_yash2", "_type": "_doc", "_id": "r_aza5w1"}}
{"title": "Absolutely beautiful", "author": "Frocharocha", "subreddit": "Animemes", "num_comments": 72, "url": "https://i.redd.it/tsbgeq5us6l21.

{"index": {"_index": "hi_yash2", "_type": "_doc", "_id": "r_azi9ay"}}
{"title": "I Luv Kyle Lowry", "author": "jayjayzz6677", "subreddit": "torontoraptors", "num_comments": 5, "hashtag": "Kyle Lowry", "source": "reddit", "post_date": "2019-03-10T17:54:26", "upvotes": 64}
{"index": {"_index": "hi_yash2", "_type": "_doc", "_id": "r_azl7wx"}}
{"title": "Kyle Lowry 24 Pts, 10 Ast, 7 Reb Highlights vs Heat", "author": "sherryG99", "subreddit": "torontoraptors", "num_comments": 5, "hashtag": "Kyle Lowry", "source": "reddit", "post_date": "2019-03-10T22:15:17", "upvotes": 54}
{"index": {"_index": "hi_yash2", "_type": "_doc", "_id": "r_azltxk"}}
{"title": "Kyle Lowry full highlights vs Miami Heat - 24 Pts, 10 Asts, 6-11 3PM, +31", "author": "Bowiescorvat2", "subreddit": "nba", "num_comments": 11, "hashtag": "Kyle Lowry", "source": "reddit", "post_date": "2019-03-10T23:12:42", "upvotes": 26}
{"index": {"_index": "hi_yash2", "_type": "_doc", "_id": "r_az9qkd"}}
{"title": "What are you All- Defen

{"index": {"_index": "hi_yash2", "_type": "_doc", "_id": "r_azecjp"}}
{"title": "[Share] Life Response Frequencies by Jeffrey Gignac", "author": "adiruti", "subreddit": "downloadass", "num_comments": 0, "hashtag": "Gignac", "source": "reddit", "post_date": "2019-03-10T10:12:17", "upvotes": 1}
{"index": {"_index": "hi_yash2", "_type": "_doc", "_id": "r_azbll6"}}
{"title": "Nunca dejen de decirle chau", "author": "the_son_of_di_fiori", "subreddit": "argentina", "num_comments": 136, "hashtag": "Totalmente", "source": "reddit", "post_date": "2019-03-10T03:23:28", "upvotes": 1117}
{"index": {"_index": "hi_yash2", "_type": "_doc", "_id": "r_az9kpq"}}
{"title": "Qu\u00e9 estudiar para ayudar a gente sin techo", "author": "OuyiiDEXX", "subreddit": "chile", "num_comments": 28, "hashtag": "Totalmente", "source": "reddit", "post_date": "2019-03-09T23:31:50", "upvotes": 11}
{"index": {"_index": "hi_yash2", "_type": "_doc", "_id": "r_azit4v"}}
{"title": "Eu deveria voltar a ir ao cinema...", "autho

{"index": {"_index": "hi_yash2", "_type": "_doc", "_id": "r_azeypk"}}
{"title": "Got at least another solid use in it yet", "author": "you-want-nodal", "subreddit": "ScottishPeopleTwitter", "num_comments": 83, "url": "https://i.redd.it/0van2b0v5al21.jpg", "hashtag": "Scotland", "source": "reddit", "post_date": "2019-03-10T11:53:03", "upvotes": 6544}
{"index": {"_index": "hi_yash2", "_type": "_doc", "_id": "r_azdkr4"}}
{"title": "Police brutality l", "author": "Unknockable", "subreddit": "PublicFreakout", "num_comments": 165, "hashtag": "Scotland", "source": "reddit", "post_date": "2019-03-10T07:58:21", "upvotes": 427}
{"index": {"_index": "hi_yash2", "_type": "_doc", "_id": "r_azftpm"}}
{"title": "Not_playing_in_Europe.jpg", "author": "JustThePanos", "subreddit": "eu4", "num_comments": 79, "url": "https://i.redd.it/ngi71govqal21.jpg", "hashtag": "Scotland", "source": "reddit", "post_date": "2019-03-10T13:51:35", "upvotes": 1008}
{"index": {"_index": "hi_yash2", "_type": "_doc", "_id": 

...

{"index": {"_index": "hi_yash2", "_type": "_doc", "_id": "r_azandp"}}
{"title": "I met Brie Larson working the night shift at my local AMC", "author": "sadokistpotato", "subreddit": "marvelstudios", "num_comments": 3483, "url": "https://i.imgur.com/mmwkl3g.jpg", "hashtag": "Brie Larson", "source": "reddit", "post_date": "2019-03-10T01:30:23", "upvotes": 77371}
{"index": {"_index": "hi_yash2", "_type": "_doc", "_id": "r_azi435"}}
{"title": "I saw on Reddit that a guy, met a guy who met Brie Larson working the night shift at his local AMC", "author": "thecallumread", "subreddit": "marvelstudios", "num_comments": 90, "url": "https://i.redd.it/3t73v70xvbl21.jpg", "hashtag": "Brie Larson", "source": "reddit", "post_date": "2019-03-10T17:40:59", "upvotes": 4606}
{"index": {"_index": "hi_yash2", "_type": "_doc", "_id": "r_azk6xb"}}
{"title": "CMV: Rotten Tomatoes/Disney are justified in removing bad reviews of Captain Marvel, and the outrage over Brie Larson makes no sense", "author": "Inmoni

{"index": {"_index": "hi_yash2", "_type": "_doc", "_id": "r_azllhc"}}
{"title": "Soy colombiano y tengo una pregunta: \u00bfCreen que el apag\u00f3n de hoy d\u00e9 luz verde a eventos mayores, a un punto de no retorno?", "author": "plamarsh", "subreddit": "vzla", "num_comments": 5, "hashtag": "ojal\u00e1", "source": "reddit", "post_date": "2019-03-10T22:49:37", "upvotes": 2}
{"index": {"_index": "hi_yash2", "_type": "_doc", "_id": "r_azec7b"}}
{"title": "As a Northerner, seeing the bill for breakfast at a cafe in Kensington sent me into a mild panic attack.", "author": "The_Grizzly_Bear", "subreddit": "britishproblems", "num_comments": 737, "hashtag": "London", "source": "reddit", "post_date": "2019-03-10T10:10:45", "upvotes": 6394}
{"index": {"_index": "hi_yash2", "_type": "_doc", "_id": "r_azi9ze"}}
{"title": "London has fallen", "author": "jimboconker77", "subreddit": "The_Donald", "num_comments": 418, "url": "https://i.redd.it/4s31avumybl21.jpg", "hashtag": "London", "source": "red

{"index": {"_index": "hi_yash2", "_type": "_doc", "_id": "r_azgfzq"}}
{"title": "Endless War", "author": "glorious_bangla", "subreddit": "PoliticalHumor", "num_comments": 1567, "url": "https://i.redd.it/b3yfjyp93bl21.jpg", "hashtag": "United", "source": "reddit", "post_date": "2019-03-10T15:00:17", "upvotes": 36627}
{"index": {"_index": "hi_yash2", "_type": "_doc", "_id": "r_azh6jb"}}
{"title": "[Match Thread] Arsenal vs. Manchester United", "author": "curlyswine", "subreddit": "reddevils", "num_comments": 5952, "hashtag": "United", "source": "reddit", "post_date": "2019-03-10T16:14:00", "upvotes": 210}
{"index": {"_index": "hi_yash2", "_type": "_doc", "_id": "r_az9cl7"}}
{"title": "Which word appears most often on each country\u2019s English language Wikipedia page", "author": "halleyadaptable", "subreddit": "MapPorn", "num_comments": 827, "url": "https://i.redd.it/igvugaq3d6l21.png", "hashtag": "United", "source": "reddit", "post_date": "2019-03-09T23:07:55", "upvotes": 13338}
{"inde

{"index": {"_index": "hi_yash2", "_type": "_doc", "_id": "r_azhxp4"}}
{"title": "Este verano puse una c\u00e1mara trampa en el campo de mis viejos (X regi\u00f3n) y aparecieron pumas chicos y gorditos.", "author": "atenux", "subreddit": "chile", "num_comments": 22, "hashtag": "Pumas", "source": "reddit", "post_date": "2019-03-10T17:24:30", "upvotes": 183}
{"index": {"_index": "hi_yash2", "_type": "_doc", "_id": "r_azi0n0"}}
{"title": "Match Thread: Pumas UNAM vs Morelia [Liga BBVA Bancomer]", "author": "LiveCommentator", "subreddit": "LigaMX", "num_comments": 53, "hashtag": "Pumas", "source": "reddit", "post_date": "2019-03-10T17:32:07", "upvotes": 9}
{"index": {"_index": "hi_yash2", "_type": "_doc", "_id": "r_aze36m"}}
{"title": "Me and my dad picked up this 850 T5 for \u00a3400 today!", "author": "assass1nat0r", "subreddit": "projectcar", "num_comments": 5, "url": "https://i.redd.it/wtfqdf4sf9l21.png", "hashtag": "Pumas", "source": "reddit", "post_date": "2019-03-10T09:26:52", "upvot

{"index": {"_index": "hi_yash2", "_type": "_doc", "_id": "r_az9k87"}}
{"title": "Game Thread: Toronto Maple Leafs (41-21-5) at Edmonton Oilers (30-30-7) - 09 Mar 2019 - 05:00PM MST", "author": "GDT_Bot", "subreddit": "hockey", "num_comments": 1846, "hashtag": "Edmonton", "source": "reddit", "post_date": "2019-03-09T23:30:28", "upvotes": 137}
{"index": {"_index": "hi_yash2", "_type": "_doc", "_id": "r_azbevd"}}
{"title": "Asked what the difference was tonight McDavid said \u201cThey\u2019re just... they\u2019re a good hockey team, they are solid all over, they never make mistakes, obviously skilled, they just come at you in waves. \u201c", "author": "rsyzygy", "subreddit": "leafs", "num_comments": 50, "hashtag": "Edmonton", "source": "reddit", "post_date": "2019-03-10T03:00:57", "upvotes": 227}
{"index": {"_index": "hi_yash2", "_type": "_doc", "_id": "r_azgdb1"}}
{"title": "In case you ever wondered \"Do hockey players actually talk like that?\"", "author": "ParacelsusPATBH", "subreddit

In [ ]:
# This is a working example: make sure the indexes match
import json
data = [
    { "index" : { "_index" : "test5", "_type" : "type1", "_id" : "1" } },
    { "field1" : "value1" },
    { "delete" : { "_index" : "test5", "_type" : "type1", "_id" : "2" }, },
    { "create" : { "_index" : "test5", "_type" : "type1", "_id" : "3" }, },
    { "field1" : "value3" },
    { "update" : {"_id" : "1", "_type" : "type1", "_index" : "test5"} },
    { "doc" : {"field2" : "value2"} }
]

data_to_post = '\n'.join(json.dumps(d) for d in data)
print(data_to_post)
#r = requests.post('http://34.73.60.209:9200/test5/_bulk', headers = headers, data=data_to_post + '\n')
#print(r.text)

In [9]:
from datetime import datetime
datetime.utcfromtimestamp(1552252748)
# Communicating with Elastic Search:
#requests.post('http://34.73.60.209:9200/hi_yash/_doc/',  json = {"user" : "mchifala",
  #  "post_date" : "2019-03-10T15:41:12",
  #  "message" : "Hi Yash"})


datetime.datetime(2019, 3, 10, 21, 19, 8)